In [1]:
## necessary packages

using Distributions
using Distances
using LinearAlgebra
using SparseArrays
using IterativeSolvers
using ProgressMeter
using JLD2
using Random
using SpecialFunctions # Matern functions
using MLBase         # cross-validation 
using Convex         # compute stacking weights
using MosekTools     # compute stacking weights
include("./utils.jl")

sp_stacking_K_fold_MT (generic function with 1 method)

In [2]:
# Set the parameters of the simulated data #
N_ls = [200, 300, 400, 500, 600, 700, 800, 900];
N_ho = 100;
seed = 1;
time_compar = Array{Float64, 3}(undef, length(N_ls), 3, 2);

In [3]:
## candidate values of hyperparameters for stacking ##
p = 2
deltasq_grid = [0.1, 0.5, 1, 2];
phi_grid = [3, 9, 15, 21];
nu_grid = [0.5, 1, 1.5, 1.75];
## priors parameters ##
Priors = Dict("μβ" => fill(0.0, p), "inv_V_β" => Diagonal(ones(p) * 0.25), 
    "aσ" => 2.0, "bσ" => 2.0);
K_fold = 10;

In [23]:
for sim_id in 1:2
    print("\n", "simulation: ", sim_id, "\n");
    if sim_id == 1
        p = 2;      # No. covariates
        β = [1.0 2.0]; #regression coeff
        σ2 = 1.0; ϕ = 7.0; ν = 1.0; τ2 = 1.0; # hyperparmeters in matern
    else
        p = 2;      # No. covariates
        β = [1.0 2.0]; #regression coeff
        σ2 = 1.0; ϕ = 20.0; ν = 0.5; τ2 = 0.3; # hyperparmeters in matern
    end
    for i in 1:2 #length(N_ls)
        print("dataset: ", i, "\n")
        
        # generate simulation data #
        
        Random.seed!(i);
        N = N_ls[i];             # No. all positions
        ind_mod = 1:(N - N_ho);      # index of training observations
        coords = rand(2, N);         # random location over unit square (2 by N)
        X = vcat(fill(1.0, (1, N)), rand(Normal(), (1, N)));          # design matrix (p by N)
        D = pairwise(Euclidean(), coords, dims = 2);                  # distance matrix
        Cov = Symmetric(Maternlu.(UpperTriangular(D), 
                ν = ν, ϕ = ϕ, σ2 = σ2))                               # covariance matrix
        z = rand(MvNormal(Cov), 1);                                   # latent process
        y = (β * X)[1,:] + z[:,1] + sqrt(τ2) * rand(Normal(), N);     # response

        ## fit stacking ##
        # need to compile the function first to obtain precise time estimation #
        if sim_id == 1 & i == 1 
            time_compar[i, 1, sim_id] = @elapsed sp_stacking_K_fold_MT(X[:, ind_mod], y[ind_mod], coords[:, ind_mod], 
                deltasq_grid, phi_grid, nu_grid, Priors, K_fold = K_fold, 
                seed = seed, label = "LSE");
            time_compar[i, 2, sim_id] = @elapsed sp_stacking_K_fold_MT(X[:, ind_mod], y[ind_mod], coords[:, ind_mod], 
                deltasq_grid, phi_grid, nu_grid, Priors, K_fold = K_fold, 
                seed = seed, label = "LP");
        end

        time_compar[i, 1, sim_id] = @elapsed sp_stacking_K_fold_MT(X[:, ind_mod], y[ind_mod], coords[:, ind_mod], 
            deltasq_grid, phi_grid, nu_grid, Priors, K_fold = K_fold, 
            seed = seed, label = "LSE");

        time_compar[i, 2, sim_id] = @elapsed sp_stacking_K_fold_MT(X[:, ind_mod], y[ind_mod], coords[:, ind_mod], 
            deltasq_grid, phi_grid, nu_grid, Priors, K_fold = K_fold, 
            seed = seed, label = "LP");
        
        ## fit MCMC ##
        inv_V_beta = Priors["inv_V_β"]
        using RCall
        @rput X
        @rput y
        @rput ind_mod
        @rput coords
        @rput inv_V_beta
        R"""

        library(spBayes)
        library(ggplot2)
        library(geoR)

        X = t(X);
        coords = t(coords);

        n.samples <- 200
        starting <- list("phi"=3/0.5, "sigma.sq"=1, "tau.sq"=1, "nu" = 0.5)
        tuning <- list("phi"=0.1, "sigma.sq"=0.1, "tau.sq"=0.1, "nu" = 0.1)
        priors.1 <- list("beta.Norm"=list(rep(0, ncol(X)), solve(inv_V_beta)),
                       "phi.Unif"=c(3, 21), "sigma.sq.IG"=c(2, 2),
                       "tau.sq.IG"=c(2, 2), "nu.unif" = c(0.25, 2))

        cov.model <- "matern"
        n.report <- 50
        verbose <- TRUE
        m.1 <- spLM(y[ind_mod]~X[ind_mod, ]-1, coords=coords[ind_mod, ],
                  starting=starting,
                  tuning=tuning, priors=priors.1, cov.model=cov.model,
                  n.samples=n.samples, verbose=verbose, n.report=n.report)

        MCMC_t <- m.1$run.time[3]

        """
        @rget MCMC_t
        
        time_compar[i, 3, sim_id] = MCMC_t
    end  
end


simulation: 1
dataset: 1	----------------------------------------
	General model description
----------------------------------------
Model fit with 100 observations.

Number of covariates 2 (including intercept if specified).

Using the matern spatial correlation model.

Number of MCMC samples 200.

Priors and hyperpriors:
	beta normal:
	mu:	0.000	0.000	
	cov:
	4.000	0.000	
	0.000	4.000	

	sigma.sq IG hyperpriors shape=2.00000 and scale=2.00000
	tau.sq IG hyperpriors shape=2.00000 and scale=2.00000
	phi Unif hyperpriors a=3.00000 and b=21.00000
	nu Unif hyperpriors a=0.25000 and b=2.00000
-------------------------------------------------
		Sampling
-------------------------------------------------
Sampled: 50 of 200, 25.00%
Report interval Metrop. Acceptance rate: 50.00%
Overall Metrop. Acceptance rate: 50.00%
-------------------------------------------------
Sampled: 100 of 200, 50.00%
Report interval Metrop. Acceptance rate: 48.00%
Overall Metrop. Acceptance rate: 49.00%
----------

In [24]:
time_compar[:,:, :]

8×3×2 Array{Float64, 3}:
[:, :, 1] =
 0.146549  0.276442  0.899
 0.627299  0.711668  3.134
 0.0       0.0       0.0
 0.0       0.0       0.0
 0.0       0.0       0.0
 0.0       0.0       0.0
 0.0       0.0       0.0
 0.0       0.0       0.0

[:, :, 2] =
 0.204224  0.185996  0.548
 0.568095  0.636111  3.546
 0.0       0.0       0.0
 0.0       0.0       0.0
 0.0       0.0       0.0
 0.0       0.0       0.0
 0.0       0.0       0.0
 0.0       0.0       0.0

Copies variables from R to Julia using the same name.
